# Wine aroma prediction using CNNs
Here we use CNN to predict wine aroma from the previously obtained wine composition matrices

In [ ]:
#Import the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torchvision as tv
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from torch.cuda.amp import autocast, GradScaler
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
import pandas as pd 

## 1. Working with data
At this stage, we load the previously prepared data, and process the data for further neural networking 

In [537]:
# Loading data
X_array =  np.load('X_array.npy')
Y_array =  np.load('Y_array.npy')

In [538]:
# Separation of data into training, validation and test data in the ratio of 70:20:10
X_train, X_test, y_train, y_test = train_test_split(X_array, Y_array, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [539]:
# Conversion to a tensor
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_val = torch.tensor(X_val)
y_val = torch.tensor(y_val)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [540]:
print(len(X_train))
print(len(y_train))
print(len(X_val))
print(len(y_val))
print(len(X_test))
print(len(y_test))

323
323
81
81
45
45


In [541]:
# Create a custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [542]:
# Create Dataset and DataLoader
dataset_train = CustomDataset(X_train, y_train)
dataset_val = CustomDataset(X_val, y_val)
dataset_test = CustomDataset(X_test, y_test)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=32, shuffle=False)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=False)

## 2. CNN architecture
At this step we design the architecture of the neural network, in this file the optimal architecture is specified

In [543]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Define convolution layers with batch normalization
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        # Pulling
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 11 * 25, 512)  
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
        
        # Dropout to prevent overtraining
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
 
        # Straightening before feeding to the full-link layer
        x = x.view(-1, 128 * 11 * 25)  # Flatten
        
        # Fully connected layers with activation and dropout
        x = F.leaky_relu(self.fc1(x))
        x = self.dropout(x)  # Dropout after the first full-link layer
        
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    

# Creating an instance of a neural network
net = ConvNet()
# Print the architecture of the neural network
print(net)

ConvNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=35200, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


## 3. Neural network training
Set the training parameters and train the neural network

In [ ]:
loss_fn = nn.BCEWithLogitsLoss() 
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=1e-5) 
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma = 0.6
)

In [545]:
# Accuracy function
def accuracy(pred, label, threshold=0.5):
    pred = torch.sigmoid(pred)
    pred_labels = (pred > threshold).float()  # Threshold the predictions
    correct = (pred_labels == label).sum().item()  # Compare predictions with labels
    total = label.size(0) * label.size(1)  # Total number of labels
    return correct / total

In [546]:
# Transferring the neural network to cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = net.to(device)
loss_fn = loss_fn.to(device)

In [547]:
use_amp = True
scaler = torch.cuda.amp.GradScaler()
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

C:\Users\V\AppData\Local\Temp\ipykernel_5124\2820173606.py:2: FutureWarning:

`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.



In [ ]:
# Neural network training
epochs = 100 

for epoch in range(epochs):
    loss_val = 0.0
    acc_val = 0.0
    for sample in tqdm(dataloader_train):
        matrix, label = sample[0].to(device), sample[1].to(device) #Transferring to cuda
        matrix = matrix.float()
        optimizer.zero_grad()

        # Let's add channel measurement to the data
        matrix = matrix.unsqueeze(1)
  
        with autocast(use_amp):
            pred = model(matrix)
            loss = loss_fn(pred, label.float()) 

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        # Accuracy calculation
        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss_val/len(dataloader_train):.5f}, Accuracy: {acc_val/len(dataloader_train):.3f}')

  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\V\AppData\Local\Temp\ipykernel_5124\3629959804.py:13: FutureWarning:

`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.

100%|██████████| 11/11 [00:00<00:00, 19.00it/s]


Epoch: [1/100], Loss: 0.56558, Accuracy: 0.739


100%|██████████| 11/11 [00:00<00:00, 41.37it/s]


Epoch: [2/100], Loss: 0.46993, Accuracy: 0.791


100%|██████████| 11/11 [00:00<00:00, 42.30it/s]


...

Epoch: [98/100], Loss: 0.34349, Accuracy: 0.847


100%|██████████| 11/11 [00:00<00:00, 45.54it/s]


Epoch: [99/100], Loss: 0.32506, Accuracy: 0.856


100%|██████████| 11/11 [00:00<00:00, 45.62it/s]

Epoch: [100/100], Loss: 0.32920, Accuracy: 0.853


## 4. Checking on the validation dataset

In [549]:
loss_val = 0.0
acc_val = 0.0
for sample in tqdm(dataloader_val):
    matrix, label = sample[0].to(device), sample[1].to(device)
    matrix = matrix.float()
    # Add channel measurement to the data
    matrix = matrix.unsqueeze(1)
  
    with autocast(use_amp):
        pred = model(matrix)
        loss = loss_fn(pred, label)

    loss_item = loss.item()
    loss_val += loss_item


    acc_current = accuracy(pred.cpu().float(), label.cpu().float())
    acc_val += acc_current


print(f'Loss: {loss_val/len(dataloader_val):.5f}, Accuracy: {acc_val/len(dataloader_val):.3f}')

  0%|          | 0/3 [00:00<?, ?it/s]C:\Users\V\AppData\Local\Temp\ipykernel_5124\4114307137.py:9: FutureWarning:

`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.

100%|██████████| 3/3 [00:00<00:00, 120.04it/s]

Loss: 0.58933, Accuracy: 0.779
